# Imports and setup

### Auto-re-import python modules, useful for editing local fils

In [2]:
%load_ext autoreload
%autoreload 2

## Imports

In [3]:
import itertools

import matplotlib as mpl
import polars as pl
import seaborn as sns

# pl.Config.set_verbose(True)
# pl.Config.()
pl.Config.set_streaming_chunk_size(1000)
mpl.rcParams["figure.max_open_warning"] = 0

# Handwritten local modules
# import process_scop_sourmash_multisearch_polars
# from process_scop_sourmash_multisearch_polars_mem_optimized import MultisearchParser
from process_scop_sourmash_multisearch_polars import MultisearchParser

In [4]:
pl.__version__

'1.12.0'

# Read in data

In [5]:
analysis_outdir = (
    "s3://seanome-kmerseek/scope-benchmark/analysis-outputs/2024-10-09__protein_k5-20"
)

pipeline_outdir = (
    "s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-08__protein_k5-20"
)
analysis_outdir

's3://seanome-kmerseek/scope-benchmark/analysis-outputs/2024-10-09__protein_k5-20'

## Read metadata

### Read Query Metadata

In [6]:
query_metadata = pl.scan_parquet(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.query_metadata.pq",
    low_memory=True,
)
query_metadata

/home/ec2-user/miniconda3/envs/2024-kmerseek-analysis-polars/lib/python3.12/site-packages/IPython/core/formatters.py:347: UserWarning: '(default_)region' not set; polars will try to get it from bucket

Set the region manually to silence this warning.
  return method()


### Read match metadata

In [7]:
match_metadata = pl.scan_parquet(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.match_metadata.pq",
    low_memory=True,
)
match_metadata

In [ ]:
# Skip ksizes 5 and 6 for now because those files are enormous, 2.3 GiB for k=5 and 175 MiB for k=6
# # -> Figure out how to use polars later
# ksizes = range(7, 21)
# moltype = "protein"

moltype_info = {
    # "protein": dict(
    #     ksizes=range(5, 21),
    #     pipeline_outdir="s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-08__protein_k5-20",
    #     analysis_outdir="s3://seanome-kmerseek/scope-benchmark/analysis-outputs/protein",
    # ),
    # "dayhoff": dict(
    #     ksizes=range(5, 21),
    #     pipeline_outdir="s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__dayhoff_k5-20",
    #     analysis_outdir="s3://seanome-kmerseek/scope-benchmark/analysis-outputs/dayhoff",
    # ),
    "hp": dict(
        ksizes=range(10, 20),
        # pipeline_outdir="s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60",
        pipeline_outdir="/home/ec2-user/data/seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60",
        # analysis_outdir="s3://seanome-kmerseek/scope-benchmark/analysis-outputs/hp",
        analysis_outdir="/home/ec2-user/data/seanome-kmerseek/scope-benchmark/analysis-outputs/hp",
    ),
}

for moltype, info in moltype_info.items():
    ksizes = info["ksizes"]
    analysis_outdir = info["analysis_outdir"]
    pipeline_outdir = info["pipeline_outdir"]
    for ksize in ksizes:
        # try:
        parser = MultisearchParser(
            query_metadata=query_metadata,
            match_metadata=match_metadata,
            pipeline_outdir=pipeline_outdir,
            moltype=moltype,
            ksize=ksize,
            analysis_outdir=analysis_outdir,
            verbose=True,
            input_filetype="pq",
            lazy=True,
        )
        lf = parser.process_multisearch_scop_results()
    #     break
    # break
    # lf.head().collect()
    # except FileNotFoundError:
    #     pass

    # break
lf

2024-10-31 23:13 - DEBUG - 
--- moltype: hp, ksize: 10 --

--- moltype: hp, ksize: 10 --
2024-10-31 23:13 - DEBUG - Reading /home/ec2-user/data/seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60/sourmash/multisearch/astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.10.multisearch.pq ...
Reading /home/ec2-user/data/seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60/sourmash/multisearch/astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.10.multisearch.pq ...
2024-10-31 23:13 - DEBUG - Done.
Done.
2024-10-31 23:13 - DEBUG - Removing self-matches and matches with only one hash
Removing self-matches and matches with only one hash
2024-10-31 23:13 - DEBUG - Done.
Done.
2024-10-31 23:13 - DEBUG - 
Joining multisearch with query and match metadata ...

Joining multisearch with query and match metadata ...
2024-1

In [ ]:
%debug